In [1]:
import model_lib
import numpy as np
import warnings

warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)

import os
import time
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 16
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 8
    PIN_MEMORY = True
    DATA_ORDER = "cw_ins"
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    
    # only flips
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    WIDTH = 224
    HEIGHT = 224
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
train_image_dir = "/media/Data1/interns/aravind/train2017/"
# train_image_dir = "/media/Data1/interns/aravind/val2017/"
val_image_dir = "/media/Data1/interns/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/media/Data1/interns/aravind/re/data/train_cwid.pickle"
# train_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"
val_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"

In [4]:
with open(train_pickle,"rb") as train_ann:
    train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [5]:
train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)

In [6]:
import torch.optim as optim
import torch
import torch.nn.functional as F
net = model_lib.MultiHGModel()

In [7]:
# net.vgg0.load_state_dict(torch.load("./models/split_vgg16_features_4.pt"))
# net.vgg1.load_state_dict(torch.load("./models/split_vgg16_features_1.pt"))
pretrained_dict = torch.load(model_dir+"low_11.pt")
net_dict = net.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}

net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

In [8]:
def set_trainable(module,state):
    for param in module.parameters():
        param.requires_grad = state

set_trainable(net,False)
set_trainable(net.mp0,True)
set_trainable(net.mp1,True)
set_trainable(net.class_predictor,True)
for name,child in net.vgg0.named_children():
    if name[:-1] == "layer":
        [set_trainable(s.ignore_filters,True) for s in child[::2]]
        [set_trainable(s.copy_filters,True) for s in child[::2]]
for name,child in net.vgg1.named_children():
    if name[:-1] == "layer":
        [set_trainable(s.ignore_filters,True) for s in child[::2]]
        [set_trainable(s.copy_filters,True) for s in child[::2]]


In [9]:
# optimizer = optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),lr = 0.001)
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=0.01, momentum=0.9)
param_lr = []
param_lr.append({'params': net.mp0.parameters(),'lr':1e-4,'momentum':0.9})
param_lr.append({'params': net.mp1.parameters(),'lr':1e-4,'momentum':0.9})
param_lr.append({'params': net.class_predictor.parameters(),'lr':1e-4,'momentum':0.9})

for name,child in net.vgg0.named_children():
    if name[:-1] == "layer":
        for s in child[::2]:
            lr = 0
            if int(name[-1]) > 0:
                lr = 1e-4
            else:
                lr = 0
            param_lr.append({'params':s.ignore_filters.parameters(),'lr':lr,'momentum':0.9})
            param_lr.append({'params':s.copy_filters.parameters(),'lr':lr,'momentum':0.9})
for name,child in net.vgg1.named_children():
    if name[:-1] == "layer":
        for s in child[::2]:
            lr = 0
            if int(name[-1]) > 0:
                lr = 1e-4
            else:
                lr = 0
            param_lr.append({'params':s.ignore_filters.parameters(),'lr':lr,'momentum':0.9})
            param_lr.append({'params':s.copy_filters.parameters(),'lr':lr,'momentum':0.9})

net_size = sum([i.numel() for i in net.parameters()])
trainable_params = filter(lambda p: p.requires_grad, net.parameters())
trainable_size = sum([i.numel() for i in trainable_params])
print(net_size,trainable_size)
print(param_lr)
optimizer = optim.SGD(param_lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=1, patience=10, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=100, min_lr=1e-5, eps=1e-08)

63674071 63674071
[{'params': <generator object Module.parameters at 0x7fb7ee675d58>, 'lr': 0.0001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fb7ee675f68>, 'lr': 0.0001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fb7ecb0a4c0>, 'lr': 0.0001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fb7ee6c90f8>, 'lr': 0.0001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fb7ee6c9150>, 'lr': 0.0001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fb7ee6c90a0>, 'lr': 0.0001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fb7ee699048>, 'lr': 0.0001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fb7ee6990a0>, 'lr': 0.0001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fb7ee6990f8>, 'lr': 0.0001, 'momentum': 0.9}, {'params': <generator object Module.parameters at 0x7fb7ee699150>, 'lr': 0.0001, 'momentum

In [10]:
net = net.cuda()

In [ ]:
torch.set_printoptions(threshold=10**6)
iters_per_checkpoint = 60
for epoch in range(10000):  # loop over the dataset multiple times
    running_loss = 0.0
    loss1,loss2 = 0.0,0.0
    acc1,acc2 = 0.0,0.0
    for i,data in enumerate(train_loader,0):
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = data
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_bboxes.cuda(),batch_one_hot.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        pred_class,pred_masks = net([batch_images,batch_impulses])
        # we are giving no weighting for classes...
        class_loss,mask_loss = model_lib.multi_mask_loss_criterion(pred_class,batch_one_hot,pred_masks,batch_gt_responses,batch_bboxes)
        class_acc,mask_acc = model_lib.accuracy(pred_class,batch_one_hot,pred_masks[1],batch_gt_responses)
        loss = class_loss+mask_loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item(); loss1 += class_loss.item(); loss2 += mask_loss.item()
        acc1 += class_acc.item(); acc2 += mask_acc.item()
        if i % iters_per_checkpoint == iters_per_checkpoint-1:
#             scheduler.step((loss1+loss2)/iters_per_checkpoint)
            print("batch: ",i,"epoch: ",epoch, "loss: %0.5f" % (running_loss/iters_per_checkpoint))
            print("class_loss: %0.5f \t mask_loss: %0.5f"%(loss1/iters_per_checkpoint,loss2/iters_per_checkpoint))
            print("class_acc: %0.5f \t mask_acc: %0.5f"%(acc1/iters_per_checkpoint,acc2/iters_per_checkpoint))
#             torch.save(net.state_dict(),model_dir+("model_mask_vgg_%d_%d.pt")%(2,2))
            torch.save(net.state_dict(),model_dir+("low_11.pt"))
            running_loss = 0.0; loss1 = 0.0; loss2 = 0.0
            acc1 = 0.0; acc2 = 0.0
    # print("batch: %d time:%0.3f sec" %(i, end-start)); print(loss.item())
print('Finished Training')

batch:  59 epoch:  0 loss: 1.51065
class_loss: 1.05869 	 mask_loss: 0.45196
class_acc: 0.70729 	 mask_acc: 0.63095
batch:  119 epoch:  0 loss: 1.52031
class_loss: 1.08182 	 mask_loss: 0.43848
class_acc: 0.70312 	 mask_acc: 0.64151
batch:  179 epoch:  0 loss: 1.52348
class_loss: 1.08583 	 mask_loss: 0.43765
class_acc: 0.67812 	 mask_acc: 0.64467
batch:  239 epoch:  0 loss: 1.54829
class_loss: 1.09097 	 mask_loss: 0.45732
class_acc: 0.67604 	 mask_acc: 0.62822
